<a href="https://colab.research.google.com/github/justdoit0430/DM19-Lab2-Homework/blob/master/colab_first.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [0]:
from google.colab import drive
drive.mount('/content/drive')
import json
##with open('/kaggle/input/dm19-lab2-nthu/tweets_DM.json') as json_file:
#    json_data = json.load(json_file)
    
tweets = []
for line in open('/content/drive/My Drive/Colab Notebooks/tweets_DM.json', 'r'):
    tweets.append(json.loads(line))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from pandas.io.json import json_normalize 
tdf=json_normalize(tweets)

In [0]:
tdf.rename(columns={'_source.tweet.text': 'text', '_source.tweet.tweet_id': 'tweet_id', '_source.tweet.hashtags':'Hashtags'}, inplace=True)
di = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data_identification.csv")
emo = pd.read_csv("/content/drive/My Drive/Colab Notebooks/emotion.csv")
tdf.head()

,_score,_index,_crawldate,_type,Hashtags,tweet_id,text
0,391,hashtag_tweets,2015-05-23 11:42:47,tweets,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ..."
1,433,hashtag_tweets,2016-01-28 04:52:09,tweets,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #..."
2,232,hashtag_tweets,2017-12-25 04:39:20,tweets,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k..."
3,376,hashtag_tweets,2016-01-24 23:53:05,tweets,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>
4,989,hashtag_tweets,2016-01-08 17:18:59,tweets,[],0x2de201,"""Trust is not the same as faith. A friend is s..."


In [0]:
tdf_merge = tdf.merge(emo, on='tweet_id', how='right')
tdf_merge.head()

,_score,_index,_crawldate,_type,Hashtags,tweet_id,text,emotion
0,391,hashtag_tweets,2015-05-23 11:42:47,tweets,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...",anticipation
1,433,hashtag_tweets,2016-01-28 04:52:09,tweets,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",sadness
2,376,hashtag_tweets,2016-01-24 23:53:05,tweets,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,fear
3,120,hashtag_tweets,2015-06-11 04:44:05,tweets,"[authentic, LaughOutLoud]",0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,joy
4,1021,hashtag_tweets,2015-08-18 02:30:07,tweets,[],0x2c91a8,Still waiting on those supplies Liscus. <LH>,anticipation


In [0]:
tdf_merge.shape

(1455563, 8)

In [0]:
tdf_train= tdf_merge[['tweet_id', 'text', 'emotion']] 
print(tdf_train.head())
test_merge = di.merge(tdf, on='tweet_id', how='right')
test_merge = test_merge[test_merge.identification != 'train']
test_merge.loc[:,'emotion'] = ''

   tweet_id                                               text       emotion
0  0x376b20  People who post "add me on #Snapchat" must be ...  anticipation
1  0x2d5350  @brianklaas As we see, Trump is dangerous to #...       sadness
2  0x1cd5b0                Now ISSA is stalking Tasha 😂😂😂 <LH>          fear
3  0x1d755c  @RISKshow @TheKevinAllison Thx for the BEST TI...           joy
4  0x2c91a8       Still waiting on those supplies Liscus. <LH>  anticipation


In [0]:
test_merge = test_merge[['tweet_id', 'text', '_type']] 
test_merge.rename(columns={'_type': 'emotion'}, inplace=True)
test_merge.loc[:,'emotion'] = ''
test_merge.head()

,tweet_id,text,emotion
0,0x28cc61,@Habbo I've seen two separate colours of the e...,
3,0x2db41f,@FoxNews @KellyannePolls No serious self respe...,
15,0x2466f6,"Looking for a new car, and it says 1 lady owne...",
23,0x23f9e9,@cineworld “only the brave” just out and fount...,
31,0x1fb4e1,Felt like total dog 💩 going into open gym and ...,


In [0]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
import string
import re
from ipykernel import kernelapp as app
import sys
string.punctuation
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

tdf_train['Tweet_punct'] = tdf_train['text'].apply(lambda x: remove_punct(x))
string.punctuation

def tokenization(text):
    text = re.split('\W+', text)
    return text

tdf_train['tokenized'] = tdf_train['Tweet_punct'].apply(lambda x: tokenization(x.lower()))

stopword = nltk.corpus.stopwords.words('english')

def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
tdf_train['nonstop'] = tdf_train['tokenized'].apply(lambda x: remove_stopwords(x))

ps = nltk.PorterStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

tdf_train['stemmed'] = tdf_train['nonstop'].apply(lambda x: stemming(x))

wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

tdf_train['lemmatized'] = tdf_train['nonstop'].apply(lambda x: lemmatizer(x))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stab

In [0]:
def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    tokens = re.split('\W+', text_rc)    # tokenization
    text = [ps.stem(word) for word in tokens if word not in stopword]  # remove stopwords and stemming
    return text

In [0]:
# # Import libraries
# import pandas as pd
# import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
# from sklearn.feature_extraction.text import CountVectorizer
# import nltk 
# import string
# import re
# %matplotlib inline

# pd.set_option('display.max_colwidth', 100)
# countVectorizer = CountVectorizer(analyzer=clean_text) 
# countVector = countVectorizer.fit_transform(tdf_train['text'])
# print('{} Number of tweets has {} words'.format(countVector.shape[0], countVector.shape[1]))
# #print(countVectorizer.get_feature_names(

In [0]:
# count_vect_df = pd.DataFrame(countVector.toarray(), columns=countVectorizer.get_feature_names())
# count_vect_df.head()

In [0]:
test_merge['Tweet_punct'] = test_merge['text'].apply(lambda x: remove_punct(x))
test_merge['tokenized'] = test_merge['Tweet_punct'].apply(lambda x: tokenization(x.lower()))
test_merge['nonstop'] = test_merge['tokenized'].apply(lambda x: remove_stopwords(x))
test_merge['stemmed'] = test_merge['nonstop'].apply(lambda x: stemming(x))
test_merge['lemmatized'] = test_merge['nonstop'].apply(lambda x: lemmatizer(x))

In [0]:
tdf_train.to_pickle("tdf_train.pkl")
tdf_train = pd.read_pickle("tdf_train.pkl")
test_merge.to_pickle("test_merge.pkl")
test_merge = pd.read_pickle("test_merge.pkl")

In [0]:
tdf_train.head()

,tweet_id,text,emotion,Tweet_punct,tokenized,nonstop,stemmed,lemmatized
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",anticipation,People who post add me on Snapchat must be deh...,"[people, who, post, add, me, on, snapchat, mus...","[people, post, add, snapchat, must, dehydrated...","[peopl, post, add, snapchat, must, dehydr, cuz...","[people, post, add, snapchat, must, dehydrated..."
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",sadness,brianklaas As we see Trump is dangerous to fre...,"[brianklaas, as, we, see, trump, is, dangerous...","[brianklaas, see, trump, dangerous, freepress,...","[brianklaa, see, trump, danger, freepress, aro...","[brianklaas, see, trump, dangerous, freepress,..."
2,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,fear,Now ISSA is stalking Tasha 😂😂😂 LH,"[now, issa, is, stalking, tasha, lh]","[issa, stalking, tasha, lh]","[issa, stalk, tasha, lh]","[issa, stalking, tasha, lh]"
3,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,joy,RISKshow TheKevinAllison Thx for the BEST TIME...,"[riskshow, thekevinallison, thx, for, the, bes...","[riskshow, thekevinallison, thx, best, time, t...","[riskshow, thekevinallison, thx, best, time, t...","[riskshow, thekevinallison, thx, best, time, t..."
4,0x2c91a8,Still waiting on those supplies Liscus. <LH>,anticipation,Still waiting on those supplies Liscus LH,"[still, waiting, on, those, supplies, liscus, lh]","[still, waiting, supplies, liscus, lh]","[still, wait, suppli, liscu, lh]","[still, waiting, supply, liscus, lh]"


In [0]:
from sklearn.naive_bayes import MultinomialNB 
from sklearn.metrics import accuracy_score 
from sklearn.naive_bayes import GaussianNB

In [0]:
type(tdf_train['stemmed'])

pandas.core.series.Series

In [0]:

test_merge= test_merge[['tweet_id', 'stemmed', 'emotion']] 
test_merge


,tweet_id,stemmed,emotion
0,0x28cc61,"[habbo, ive, seen, two, separ, colour, eleg, f...",
3,0x2db41f,"[foxnew, kellyannepol, seriou, self, respect, ...",
15,0x2466f6,"[look, new, car, say, ladi, owner, mean, need,...",
23,0x23f9e9,"[cineworld, brave, fountain, park, show, per, ...",
31,0x1fb4e1,"[felt, like, total, dog, go, open, gym, migrai...",
...,...,...,...
1867495,0x2c4dc2,"[, year, old, walk, astound, mum, look, bendi,...",
1867496,0x31be7c,"[one, week, go, inspiringvolunteeraward, cant,...",
1867500,0x1ca58e,"[got, caught, manga, hero, academia, dont, kno...",
1867515,0x35c8ba,"[speak, spoken, make, hot, ass, music, lh]",


In [0]:
# tdf_train['stemmed'] = tdf_train['stemmed'].str.encode
# tdf_train['stemmed'].head()
# test_merge['stemmed'] = test_merge['stemmed'].str.encode

In [0]:
tdf_train.head()

,tweet_id,text,emotion,Tweet_punct,tokenized,nonstop,stemmed,lemmatized
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",anticipation,People who post add me on Snapchat must be deh...,"[people, who, post, add, me, on, snapchat, mus...","[people, post, add, snapchat, must, dehydrated...","[peopl, post, add, snapchat, must, dehydr, cuz...","[people, post, add, snapchat, must, dehydrated..."
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",sadness,brianklaas As we see Trump is dangerous to fre...,"[brianklaas, as, we, see, trump, is, dangerous...","[brianklaas, see, trump, dangerous, freepress,...","[brianklaa, see, trump, danger, freepress, aro...","[brianklaas, see, trump, dangerous, freepress,..."
2,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,fear,Now ISSA is stalking Tasha 😂😂😂 LH,"[now, issa, is, stalking, tasha, lh]","[issa, stalking, tasha, lh]","[issa, stalk, tasha, lh]","[issa, stalking, tasha, lh]"
3,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,joy,RISKshow TheKevinAllison Thx for the BEST TIME...,"[riskshow, thekevinallison, thx, for, the, bes...","[riskshow, thekevinallison, thx, best, time, t...","[riskshow, thekevinallison, thx, best, time, t...","[riskshow, thekevinallison, thx, best, time, t..."
4,0x2c91a8,Still waiting on those supplies Liscus. <LH>,anticipation,Still waiting on those supplies Liscus LH,"[still, waiting, on, those, supplies, liscus, lh]","[still, waiting, supplies, liscus, lh]","[still, wait, suppli, liscu, lh]","[still, waiting, supply, liscus, lh]"


In [0]:
tdf_train_final= tdf_train[['tweet_id', 'stemmed', 'emotion']] 
tdf_train_final

,tweet_id,stemmed,emotion
0,0x376b20,"[peopl, post, add, snapchat, must, dehydr, cuz...",anticipation
1,0x2d5350,"[brianklaa, see, trump, danger, freepress, aro...",sadness
2,0x1cd5b0,"[issa, stalk, tasha, lh]",fear
3,0x1d755c,"[riskshow, thekevinallison, thx, best, time, t...",joy
4,0x2c91a8,"[still, wait, suppli, liscu, lh]",anticipation
...,...,...,...
1455558,0x321566,"[im, happi, nowond, name, show, happi, happysy...",joy
1455559,0x38959e,"[everi, circumt, id, like, thank, almighti, je...",joy
1455560,0x2cbca6,"[there, current, two, girl, walk, around, libr...",joy
1455561,0x24faed,"[ah, corpor, life, date, lh, use, rel, anachro...",joy


In [0]:
test_merge

,tweet_id,stemmed,emotion
0,0x28cc61,"[habbo, ive, seen, two, separ, colour, eleg, f...",
3,0x2db41f,"[foxnew, kellyannepol, seriou, self, respect, ...",
15,0x2466f6,"[look, new, car, say, ladi, owner, mean, need,...",
23,0x23f9e9,"[cineworld, brave, fountain, park, show, per, ...",
31,0x1fb4e1,"[felt, like, total, dog, go, open, gym, migrai...",
...,...,...,...
1867495,0x2c4dc2,"[, year, old, walk, astound, mum, look, bendi,...",
1867496,0x31be7c,"[one, week, go, inspiringvolunteeraward, cant,...",
1867500,0x1ca58e,"[got, caught, manga, hero, academia, dont, kno...",
1867515,0x35c8ba,"[speak, spoken, make, hot, ass, music, lh]",


In [0]:
# from sklearn.model_selection import train_test_split
# V_train, V_test, z_train, z_test = train_test_split(X_train, y_train, random_state=0)
# test_clf=Pipeline([
#                 ('tfidf', TfidfVectorizer()),
#                 ('nbc' , RandomForestClassifier(n_estimators=100, random_state=0)),
#             ])
# test_clf.fit(V_train, z_train)

In [0]:
### TEST
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

X_train = tdf_train_final['stemmed'].apply(lambda x: ' '.join(x))
y_train = tdf_train_final['emotion']
X_test = test_merge['stemmed'].apply(lambda x: ' '.join(x))
# y_test = test_merge['emotion']

clf=Pipeline([
                ('tfidf', TfidfVectorizer()),
                ('nbc' , LogisticRegression()),
            ])

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

# pipeline...habogietry

In [0]:
y_pred

array(['joy', 'sadness', 'joy', ..., 'anticipation', 'joy', 'joy'],
      dtype=object)

In [0]:
np.unique(y_pred)

array(['anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness',
       'surprise', 'trust'], dtype='<U12')

In [0]:
submission = pd.DataFrame({'id':test_merge['tweet_id'], 'emotion':y_pred})
submission.head()

,id,emotion
0,0x28cc61,joy
3,0x2db41f,joy
15,0x2466f6,joy
23,0x23f9e9,joy
31,0x1fb4e1,joy


In [0]:
export_csv = submission.to_csv ('result6.csv', index = False, header=True)